# Recursive data types and functions

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [4]:
object StdDefinition {
    sealed abstract class List[A]
    case class Empty[A]() extends List[A]
    case class NonEmpty[A](head: A, tail: List[A])
        extends List[A]
    
    type IntList = List[Int]
    
    val l: List[Int] = 
        NonEmpty(4, NonEmpty(5, NonEmpty(6, Empty())))
}

defined object StdDefinition

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [5]:
object ActualStdDefinition{
    sealed abstract class List[+A]{
        def tail: List[A] = this match {
            case Nil => ??? 
            case h :: t => t
        }
    }
    case object Nil extends List[Nothing]
    case class ::[A](head: A, tail: List[A]) extends List[A]
}

defined object ActualStdDefinition

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [8]:
// Less beautifully 

val l1: List[Int] = 
    ::(1, ::(2, ::(3, Nil)))
// More idiomatically

val l2: List[Int] = 
    1 :: 2 :: 3 :: Nil

val l3: List[Int] = 
    List(1,2,3)

l1: List[Int] = List(1, 2, 3)
l2: List[Int] = List(1, 2, 3)
l3: List[Int] = List(1, 2, 3)

And can also pattern match lists, similarly:

In [11]:
// Less beautifully

l1 match {
    case Nil => 0
    case ::(head, tail) => 1
}
// more idiomatically

l1 match {
    case Nil => 0
    case head :: tail => 1
}

// or

l1 match {
    case List() => 0
    case List(h1, h2, h3, h4) => 1
    case a => 2
}


res10_0: Int = 1
res10_1: Int = 1
res10_2: Int = 2

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [33]:
// Using mutable variables

def lengthI[A](list: List[A]): Int = {
    var out: Int = 0
    /*for (e <- list)
        out = out + 1*/
    var aux: List[A] = list
    while (aux != Nil) {
        out = out + 1
        // aux = aux.asInstanceOf[::[A]].tail
        aux = aux match {
            case Nil => Nil // throw new Exception("should not happen") 
            case _ :: tail => tail
        }
    }
    out
}

defined function lengthI

In [14]:
length(List(1,2,3,4,5))

res13: Int = 5

The recursive function is implemented as follows: 

In [20]:
// Using recursive functions

def length[A](list: List[A]): Int = 
    (list match {
        case Nil => ??? : Int
        case head :: tail => ??? : Int
    }) : Int


defined function length

In [23]:
// Using recursive functions

def length[A](list: List[A]): Int = 
    list match {
        case Nil => 0 : Int
        case head :: tail => 
            1 + length(tail) : Int
    }


defined function length

In [51]:
// Using recursive functions

def lengthR[A](list: List[A]): Int = 
    list match {
        case Nil => 0
        case _ :: tail => 
            1 + lengthR(tail)
    }


defined function lengthR

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [31]:
// Using tail-recursive functions

def lengthTR[A](list: List[A]): Int = {
    
    def lengthAux(out: Int, aux: List[A]): Int = 
        aux match {
            case Nil => out 
            case _ :: tail => 
                lengthAux(out + 1, tail) 
        }
    
    lengthAux(0, list)
}

defined function lengthTR

We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [30]:
// First, imperatively
List.fill(100)(0)


res29: List[Int] = List(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...

In [ ]:
// Next, tail-recursively



Now, let's calculate the length of a list long enough, using each of the three implementations:

In [37]:
// Imperatively
lengthI(List.fill(1000000)(0))

res36: Int = 1000000

In [42]:
// Tail-recursive
lengthTR(List.fill(10000000)(0))

res41: Int = 10000000

In [53]:
// Plain recursive
lengthR(List.fill(10000)(0))

: 

### Unit testing with `scalatest`

In [2]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [57]:
class TestLength(lengthF: List[Int] => Int)
    extends FlatSpec with Matchers{

    "length" should "work" in {
        lengthF(List()) shouldBe 0
        lengthF(List(1)) shouldBe 1
        lengthF(List(1,2,3,4)) shouldBe 4
    }
}


defined class TestLength

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [58]:
val testingCatalogue: TestLength = 
    new TestLength(lengthI)



testingCatalogue: TestLength = cmd56$Helper$TestLength

In [60]:
run(testingCatalogue)
run(new TestLength(lengthTR))
run(new TestLength(lengthR))

cmd56$Helper$TestLength:
length
- should work
cmd56$Helper$TestLength:
length
- should work
cmd56$Helper$TestLength:
length
- should work


### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [65]:
class TestSum(sum: List[Int] => Int) 
    extends FlatSpec with Matchers{
    "sum" should "work" in {
        
        sum(Nil) shouldBe 0
        sum(List(5)) shouldBe 5
        sum(List(1,2,3)) shouldBe 6
    }
}

defined class TestSum

In [68]:
// Recursively

def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0 : Int
        //case head :: Nil => head
        case head :: tail => 
            head + sum(tail) : Int
    }

defined function sum

In [69]:
run(new TestSum(sum))

cmd64$Helper$TestSum:
sum
- should work


In [69]:
// With tail-recursion



### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [ ]:
object TestProduct extends FlatSpec with Matchers{
    "product" should "work" in {
    }
}

This is the common recursive implementation:

It works as expected: 

In [ ]:
run(TestProduct)

But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [ ]:
/*def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail => head * product(tail)
    }
    */

In [ ]:
run(TestProduct)

A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [ ]:
object TestMember extends FlatSpec with Matchers{
    "member" should "work" in {
    }
}

In [ ]:
run(TestMember)

We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [4]:
class TestLast(last: List[Int] => Option[Int]) 
extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,2,3)) shouldBe Some(3)
        last(List(1)) shouldBe Some(1)
        last(List()) shouldBe None
    }
}

defined class TestLast

In [8]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            tail match {
                case Nil => 
                    Some(head) : Option[A]
                case head2 :: tail2 => 
                    last(tail) : Option[A]
            } 
    }

defined function last

In [8]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            tail match {
                case Nil => 
                    Some(head) : Option[A]
                case _ :: _ => 
                    last(tail) : Option[A]
            } 
    }

defined function last

In [9]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            if (tail.isEmpty) Some(head)
            else last(tail)
    }

defined function last

In [9]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: Nil => 
            Some(head) : Option[A]
        case head :: head2 :: tail2 => 
            //last(tail) : Option[A]
            last(head2 :: tail2) : Option[A]
    }

defined function last

In [9]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: Nil => 
            Some(head) : Option[A]
        case ::(head, tail) => 
            last(tail) : Option[A]            
    }

defined function last

In [13]:
def last[A]: List[A] => Option[A] = 
    (list : List[A]) => 
        list match {
            case Nil => 
                None : Option[A]
            case head :: Nil => 
                Some(head) : Option[A]
            case ::(head, tail) => 
                last(tail) : Option[A]            
        }

defined function last

In [15]:
def last[A]: List[A] => Option[A] = 
    {
        case Nil => 
            None
        case head :: Nil => 
            Some(head)
        case _ :: tail => 
            last(tail)          
    }

defined function last

In [16]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None
        case head :: Nil => 
            Some(head)
        case _ :: tail => 
            last(tail)          
    }

defined function last

In [14]:
def last[A]: List[A] => Option[A] = 
    _ match {
        case Nil => 
            None : Option[A]
        case head :: Nil => 
            Some(head) : Option[A]
        case ::(head, tail) => 
            last(tail) : Option[A]            
    }

defined function last

In [ ]:
object Std{
    sealed abstract class List[A]
    case object Nil extends List[Nothing]
    case class ::[A](head: A, tail: List[A]) extends List[A]
}

In [12]:
run(new TestLast(last))

cmd3$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [22]:
class TestInsertLast(
    insertLast: (List[Int], Int) => List[Int]) 
extends FlatSpec with Matchers{
    "insertLast" should "work" in {
        insertLast(List[Int](), 1) shouldBe 
            (List(1) : List[Int])
        insertLast(List(1), 2) shouldBe 
            (List(1,2) : List[Int])
        insertLast(List(1,2,3,4), 5) shouldBe 
            List(1,2,3,4) :+ 5
    }
}

defined class TestInsertLast

In [21]:
List(1,2,3) :+ 4

res20: List[Int] = List(1, 2, 3, 4)

In [25]:
def insertLast[A](list: List[A], a: A): List[A] = 
    list match {
        case Nil => 
            List(a) : List[A]
        case head :: tail => 
            head :: insertLast(tail, a) : List[A]
    }

defined function insertLast

In [26]:
run(new TestInsertLast(insertLast))

cmd21$Helper$TestInsertLast:
insertLast
- should work *** FAILED ***
  List() was not equal to List(1) (cmd21.sc:5)


### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [28]:
class TestReverse(
    reverse: List[Int] => List[Int]
) extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(List()) shouldBe List()
        reverse(List(1,2,3)) shouldBe List(3,2,1)
        reverse(1 :: List(2,3)) shouldBe 
            List(3,2) :+ 1
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3,2,1)) shouldBe List(1,2,3,2,1)
        reverse(1::List(2,3,2,1)) shouldBe 
            List(1,2,3,2) :+ 1
    }
}

defined class TestReverse

In [29]:
// Recursively: Really inefficient 
def reverseR[A](list: List[A]): List[A] = 
    list match {
        case Nil => 
            Nil : List[A]
        case head :: tail => 
            val reverseTail: List[A] = reverseR(tail)
            reverseTail :+ head : List[A]
    }

defined function reverseR

In [33]:
// Recursively: Really inefficient 
def reverseR[A](list: List[A]): List[A] = 
    list match {
        case Nil => 
            Nil
        case head :: tail => 
            reverseR(tail) :+ head
    }

defined function reverseR

In [30]:
run(new TestReverse(reverseR))

cmd27$Helper$TestReverse:
reverse
- should work


In [35]:
// out = Nil
// aux = 1 :: 2 :: 3 :: Nil
// 1 it.
// out = 1 :: Nil
// aux = 2 :: 3 :: Nil
// 2 it. 
// out = 2 :: (1 :: Nil)
// aux = 3 :: Nil
// 3 it. 
// out = 3 :: 2 :: 1 :: Nil
// aux = Nil

// Tail-recursive, efficiently
def reverseI[A](list: List[A]): List[A] = {
    var out: List[A] = List()
    var aux: List[A] = list
    while (aux != Nil) {
        out = aux.head :: out
        aux = aux.tail
    }
    out
}

defined function reverseI

In [37]:
// out = Nil
// aux = 1 :: 2 :: 3 :: Nil
// 1 it.
// out = 1 :: Nil
// aux = 2 :: 3 :: Nil
// 2 it. 
// out = 2 :: (1 :: Nil)
// aux = 3 :: Nil
// 3 it. 
// out = 3 :: 2 :: 1 :: Nil
// aux = Nil

// Tail-recursive, efficiently
def reverseT[A](list: List[A]): List[A] = {
    
    def revAux(out: List[A], aux: List[A]): List[A] = 
        aux match {
            case Nil => out
            case head :: tail => 
                revAux(head :: out, tail)
        }
    /*
    var out: List[A] = List()
    var aux: List[A] = list
    while (aux != Nil) {
        out = aux.head :: out
        aux = aux.tail
    }
    out*/
    revAux(List(), list)
}

defined function reverseI

In [38]:
run(new TestReverse(reverseT))

cmd27$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)